In [1]:
from datasets import load_dataset
from panns_inference import AudioTagging
from tqdm import tqdm
from IPython.display import Audio, display
import numpy as np

c:\Users\leony\Documents\Purdue\lance\vectordb-recipes\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_dataset("ashraq/esc50", split="train")
at = AudioTagging(checkpoint_path=None, device="cuda")

import lancedb
db = lancedb.connect("data/audio-lancedb")
table_name = "audio-search"

Repo card metadata block was not found. Setting CardData to empty.


Checkpoint path: C:\Users\leony\panns_data\Cnn14_mAP=0.431.pth
Using CPU.


In [13]:
def insert_audio():
    batches = [batch["audio"] for batch in dataset.iter(100)]
    meta_batches = [batch["category"] for batch in dataset.iter(100)]
    audio_data = [np.array([audio["array"] for audio in batch]) for batch in batches]
    meta_data = [np.array([meta for meta in batch]) for batch in meta_batches]
    print("Start")
    for i in tqdm(range(len(audio_data))):
        (_, embedding) = at.inference(audio_data[i])
        data = [{"audio": x[0]['array'], "vector": x[1], 'sampling_rate': x[0]['sampling_rate'], 'category': meta_data[i][j]} for j, x in enumerate(zip(batches[i], embedding))]
        if table_name not in db.table_names():
            tbl = db.create_table(table_name, data)
        else:
            tbl = db.open_table(table_name)
            tbl.add(data)

In [14]:
!rf -rf data/audio-lancedb/audio-search.lance

insert_audio()

Start


100%|██████████| 20/20 [05:09<00:00, 15.47s/it]


In [15]:
def search_audio(id):
    tbl = db.open_table(table_name)
    audio = dataset[id]['audio']['array']
    category = dataset[id]['category']
    display(Audio(audio, rate=dataset[id]['audio']['sampling_rate']))
    print("Category:", category)

    (_, embedding) = at.inference(audio[None, :])
    result = tbl.search(embedding[0]).limit(5).to_df()
    print(result)
    for i in range(len(result)):
        display(Audio(result['audio'][i], rate=result['sampling_rate'][i]))
        print("Category:", result['category'][i])

In [16]:
search_audio(400)

Category: car_horn
                                               audio  \
0  [0.04241943359375, 0.046630859375, 0.047332763...   
1  [-0.259735107421875, -0.262603759765625, -0.26...   
2  [0.032379150390625, 0.057159423828125, 0.06280...   
3  [-0.015625, -0.018829345703125, -0.02023315429...   
4  [-0.171478271484375, -0.21160888671875, -0.240...   

                                              vector  sampling_rate  category  \
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...          44100  car_horn   
1  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...          44100  car_horn   
2  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...          44100  car_horn   
3  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...          44100     siren   
4  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...          44100  car_horn   

          score  
0  8.457058e-12  
1  4.979556e+01  
2  5.002319e+01  
3  5.235638e+01  
4  5.290439e+01  


Category: car_horn


Category: car_horn


Category: car_horn


Category: siren


Category: car_horn
